In [75]:
import torch 
import torch.nn as nn
import pickle
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import os
import torch.nn.functional as F

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [9]:
with open('../Data/acoustic.pickle', 'rb') as f:
    data1 = pickle.load(f)
with open('../Data/seismic.pickle', 'rb') as f:
    data2 = pickle.load(f)
    

In [114]:
with open('../Data/combined.pickle', 'rb') as f:
    data = pickle.load(f)

In [115]:
data

,class,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,3,-0.023321,-0.023527,-0.022061,-0.022273,-0.017079,0.018815,1.000000,0.031178,-0.017721,...,-0.038098,-0.035813,-0.036718,-0.038129,-0.033562,-0.038832,-0.038019,-0.037385,-0.034283,-0.039099
1,1,-0.136421,-0.092407,-0.131769,0.574799,-0.073130,0.021133,0.099456,1.000000,0.614752,...,-0.042590,-0.045936,-0.045755,-0.045720,-0.045836,-0.045605,-0.044757,-0.046381,-0.043888,-0.045910
2,3,0.134293,0.138123,0.525289,1.000000,0.832403,0.659512,-0.047688,0.097587,0.014862,...,-0.019597,-0.046355,-0.016725,-0.110025,-0.074977,-0.020507,-0.081088,-0.117135,0.124611,-0.084323
3,3,0.712613,0.782345,0.190930,0.229841,0.560167,0.192150,1.000000,0.783759,-0.025315,...,0.063093,-0.020576,-0.054475,0.056441,-0.033763,-0.020097,-0.038449,-0.030611,0.015814,-0.029294
4,2,0.197980,0.300174,0.130691,0.802940,1.000000,0.099832,-0.009243,0.350689,0.281720,...,-0.063700,-0.047797,-0.032164,-0.056727,-0.064532,-0.039657,-0.054612,-0.004255,-0.046586,-0.053346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78818,3,-0.012863,0.012005,-0.018340,-0.009440,-0.033045,0.098008,1.000000,0.135341,0.145462,...,-0.069438,-0.023398,-0.072769,-0.036016,-0.059330,-0.054264,0.068717,-0.082765,-0.063361,-0.058121
78819,2,-0.024249,-0.024640,-0.021435,-0.038170,-0.020313,-0.011039,1.000000,0.070916,0.121450,...,-0.035993,-0.035162,-0.035777,-0.035664,-0.036003,-0.035998,-0.035592,-0.035893,-0.035913,-0.035753
78820,3,-0.059299,0.027528,-0.092111,0.653803,0.296419,1.000000,-0.040911,0.927669,-0.082818,...,-0.184531,-0.013405,0.358282,0.185890,0.470470,0.242505,0.240674,-0.109372,0.071049,0.615857
78821,3,-0.017755,0.003629,-0.001410,0.027432,0.071065,1.000000,0.003929,-0.015067,-0.013738,...,-0.027309,0.003666,-0.051606,-0.022456,0.132588,-0.101000,-0.098870,0.060842,-0.020554,-0.055325


In [10]:
data1

,class,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,2,-0.030057,-0.027216,-0.016126,0.006233,0.053416,1.000000,0.031091,0.015465,-0.013044,...,-0.029370,-0.030332,-0.023607,-0.031657,-0.028310,-0.031300,-0.031211,-0.030391,-0.030794,-0.031352
1,3,0.124642,1.000000,0.608303,0.008150,0.717687,-0.036154,-0.047996,-0.023177,-0.026854,...,-0.057607,-0.057537,-0.058274,-0.058113,-0.058398,-0.058491,-0.058231,-0.058531,-0.057697,-0.058402
2,3,-0.086143,-0.130700,-0.161727,-0.164976,0.577297,0.088806,-0.094891,1.000000,0.151082,...,-0.172071,-0.170827,-0.171194,-0.168016,-0.170520,-0.171233,-0.169749,-0.171479,-0.171045,-0.170937
3,3,-0.017615,0.021958,-0.002278,0.045736,1.000000,0.026328,-0.006924,-0.015067,0.009032,...,-0.031494,-0.031255,-0.031199,-0.030970,-0.031537,-0.031499,-0.030825,-0.031589,-0.031559,-0.031543
4,2,-0.013705,-0.021804,-0.019416,-0.017692,1.000000,-0.022303,-0.021477,-0.016256,-0.003198,...,-0.015685,-0.024433,-0.024608,-0.025368,-0.025230,-0.025162,-0.025535,-0.024619,-0.024239,-0.024822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78818,3,-0.196578,-0.190513,-0.189601,-0.190071,-0.134310,0.183397,1.000000,0.165482,0.058812,...,-0.192840,-0.195942,-0.191606,-0.193735,-0.195426,-0.195237,-0.193389,-0.194077,-0.193594,-0.195067
78819,3,-0.061304,-0.057746,-0.045711,-0.055745,0.027025,1.000000,0.027467,-0.001114,-0.034559,...,-0.057418,-0.058316,-0.053983,-0.059059,-0.059256,-0.057427,-0.058625,-0.056731,-0.053740,-0.050476
78820,3,0.919410,1.000000,0.368404,0.540824,0.348019,0.020499,-0.046345,-0.049503,-0.072119,...,-0.075835,-0.075814,-0.075791,-0.075812,-0.075693,-0.075819,-0.075829,-0.075817,-0.075885,-0.075873
78821,3,1.000000,0.135210,0.001155,0.033463,0.043519,-0.021910,0.156502,-0.017126,-0.022533,...,-0.032626,-0.032628,-0.032633,-0.032621,-0.032624,-0.032624,-0.032615,-0.032636,-0.032619,-0.032639


In [117]:
data2

,class,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,3,-0.311070,0.602284,-0.295414,-0.135353,-0.272519,0.470501,0.055555,0.350612,0.283464,...,-0.121085,0.458755,-0.287152,0.004847,-0.280585,-0.125173,0.010386,-0.288257,0.115472,0.013447
1,1,-0.036019,1.000000,0.044621,-0.017574,0.095875,-0.015582,-0.010842,-0.030141,-0.020132,...,-0.029158,-0.034659,-0.034904,-0.035061,0.006946,-0.035320,-0.032870,-0.015724,-0.032703,-0.035713
2,1,-0.102323,0.267455,0.789374,1.000000,-0.097091,-0.016171,0.186275,0.598286,-0.049404,...,-0.096673,-0.099150,-0.100942,-0.101388,-0.100168,-0.101975,-0.100745,-0.101671,-0.101034,-0.101691
3,3,-0.560520,-0.450013,0.732302,0.579498,0.405977,-0.554563,0.420540,0.075622,-0.501731,...,0.160053,0.183239,0.330863,-0.430148,-0.050102,-0.469018,-0.378305,-0.347293,-0.307555,0.100423
4,1,-0.062265,-0.059779,0.269739,0.126958,0.393020,1.000000,0.914650,-0.019336,-0.032771,...,-0.061744,-0.062252,-0.062077,-0.062227,-0.062246,-0.062236,-0.062251,-0.062113,-0.062113,-0.062244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78818,3,-0.269766,0.196617,0.072544,-0.194465,-0.109302,0.390684,-0.089140,-0.155741,1.000000,...,0.053338,0.100374,0.595054,0.099036,0.045020,0.042506,-0.061369,-0.189697,0.172433,-0.179482
78819,3,-0.027526,1.000000,-0.006795,-0.026603,-0.024834,-0.026691,-0.027017,-0.009539,-0.011659,...,-0.027411,-0.025038,-0.027022,-0.024138,-0.027477,-0.025119,-0.027264,-0.026643,-0.022561,-0.025719
78820,3,-0.119000,1.000000,-0.021042,0.098366,-0.013572,-0.012918,0.008290,0.012824,0.045739,...,-0.008321,0.119679,-0.042712,-0.104822,0.204603,0.005783,-0.094751,-0.004194,-0.043655,-0.109560
78821,1,-0.065489,-0.026753,0.904410,0.025899,0.353314,1.000000,0.005112,-0.002746,-0.028185,...,-0.061065,-0.064014,-0.063624,-0.065410,-0.065471,-0.064557,-0.064141,-0.065216,-0.062876,-0.065382


In [62]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [86]:
data1['class'] = data1['class'].apply(lambda x: 0 if x == 1 else (1 if x == 2 else 2)) 

In [87]:
data1

,class,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,1,-0.030057,-0.027216,-0.016126,0.006233,0.053416,1.000000,0.031091,0.015465,-0.013044,...,-0.029370,-0.030332,-0.023607,-0.031657,-0.028310,-0.031300,-0.031211,-0.030391,-0.030794,-0.031352
1,2,0.124642,1.000000,0.608303,0.008150,0.717687,-0.036154,-0.047996,-0.023177,-0.026854,...,-0.057607,-0.057537,-0.058274,-0.058113,-0.058398,-0.058491,-0.058231,-0.058531,-0.057697,-0.058402
2,2,-0.086143,-0.130700,-0.161727,-0.164976,0.577297,0.088806,-0.094891,1.000000,0.151082,...,-0.172071,-0.170827,-0.171194,-0.168016,-0.170520,-0.171233,-0.169749,-0.171479,-0.171045,-0.170937
3,2,-0.017615,0.021958,-0.002278,0.045736,1.000000,0.026328,-0.006924,-0.015067,0.009032,...,-0.031494,-0.031255,-0.031199,-0.030970,-0.031537,-0.031499,-0.030825,-0.031589,-0.031559,-0.031543
4,1,-0.013705,-0.021804,-0.019416,-0.017692,1.000000,-0.022303,-0.021477,-0.016256,-0.003198,...,-0.015685,-0.024433,-0.024608,-0.025368,-0.025230,-0.025162,-0.025535,-0.024619,-0.024239,-0.024822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78818,2,-0.196578,-0.190513,-0.189601,-0.190071,-0.134310,0.183397,1.000000,0.165482,0.058812,...,-0.192840,-0.195942,-0.191606,-0.193735,-0.195426,-0.195237,-0.193389,-0.194077,-0.193594,-0.195067
78819,2,-0.061304,-0.057746,-0.045711,-0.055745,0.027025,1.000000,0.027467,-0.001114,-0.034559,...,-0.057418,-0.058316,-0.053983,-0.059059,-0.059256,-0.057427,-0.058625,-0.056731,-0.053740,-0.050476
78820,2,0.919410,1.000000,0.368404,0.540824,0.348019,0.020499,-0.046345,-0.049503,-0.072119,...,-0.075835,-0.075814,-0.075791,-0.075812,-0.075693,-0.075819,-0.075829,-0.075817,-0.075885,-0.075873
78821,2,1.000000,0.135210,0.001155,0.033463,0.043519,-0.021910,0.156502,-0.017126,-0.022533,...,-0.032626,-0.032628,-0.032633,-0.032621,-0.032624,-0.032624,-0.032615,-0.032636,-0.032619,-0.032639


In [81]:
device = 'cpu'

In [88]:
X = data1[[str(x) for x in range(50)]]
y = data1['class']
X = X.values
y = y.values
# y = torch.tensor(y.values)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=34)


In [77]:
ms = MinMaxScaler()
ss = StandardScaler()

X_train = ms.fit_transform(X_train)
X_test = ms.fit_transform(X_test)
# y_train = y_train.reshape(-1, 1)
# y_test = y_test.reshape(-1, 1)

In [89]:
X_train

array([[ 0.97204102,  0.05565637, -0.03050711, ..., -0.10396622,
        -0.10708707, -0.11608528],
       [-0.06783397, -0.06778439, -0.06220271, ..., -0.06615854,
        -0.06337908, -0.00742759],
       [-0.02546111, -0.02505541, -0.0245985 , ..., -0.02555124,
        -0.024686  , -0.02069945],
       ...,
       [-0.06632779,  0.05824382,  0.08819027, ..., -0.07748754,
        -0.07818329, -0.07673419],
       [-0.03275923, -0.02356955, -0.0218401 , ..., -0.03407454,
        -0.02735483, -0.03268085],
       [-0.02716259,  0.0521438 ,  0.0069933 , ..., -0.03735497,
        -0.03803071, -0.0374999 ]])

In [90]:
class customDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.len = len(self.X)
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    def __len__(self):
        return self.len

In [91]:
train_data = customDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
test_data = customDataset(torch.FloatTensor(X_test), torch.LongTensor(y_test))

train_loader = DataLoader(train_data, batch_size = 64, shuffle= True)
test_loader = DataLoader(test_data, batch_size = 64, shuffle= True)

In [92]:
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel,self).__init__()
        self.fc1 = nn.Linear(50, 32)
        self.fc2 = nn.Linear(32,32)
        self.fc3 = nn.Linear(32,3)
        self.droput = nn.Dropout(0.2)

    def forward(self,x):

        x = x.view(-1,50)
        x = F.relu(self.fc1(x))
        x = self.droput(x)
        x = F.relu(self.fc2(x))
        x = self.droput(x)
        x = self.fc3(x)

        return x

model = CustomModel().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

for epoch in tqdm(range(1000)):
    cost = 0.0

    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_loader)

    if (epoch + 1) % 100 == 0:
        print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")


# with torch.no_grad():
#     model.eval()
#     classes = {0: "acute triangle", 1: "right triangle", 2: "obtuse triangle"}
#     inputs = torch.FloatTensor([
#         [9.02, 9.77, 9.96], # 0 | acute triangle
#         [8.01, 8.08, 8.32], # 0 | acute triangle
#         [3.55, 5.15, 6.26], # 1 | right triangle
#         [3.32, 3.93, 5.14], # 1 | right triangle
#         [4.39, 5.56, 9.99], # 2 | obtuse triangle
#         [3.01, 3.08, 9.98], # 2 | obtuse triangle
#         [5.21, 5.38, 5.39], # 0 | acute triangle
#         [3.85, 6.23, 7.32], # 1 | right triangle
#         [4.16, 4.98, 8.54], # 2 | obtuse triangle
#     ]).to(device)
#     outputs = model(inputs)
    
#     print('---------')
#     print(outputs)
#     print(torch.round(F.softmax(outputs, dim=1), decimals=2))
#     print(outputs.argmax(1))
#     print(list(map(classes.get, outputs.argmax(1).tolist())))

  3%|▎         | 30/1000 [00:33<18:07,  1.12s/it]


KeyboardInterrupt: 

In [213]:
nodes = 8
input_dim = X_train.shape[1]
hidden_dim = int(input_dim/nodes)
output_dim = 3

In [215]:
class MultiClassClassifierModule(nn.Module):
    def __init__(
            self,
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            output_dim=output_dim,
            dropout=0.5,
    ):
        super(MultiClassClassifierModule, self).__init__()
        self.dropout = nn.Dropout(dropout)

        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, X, **kwargs):
        X = F.relu(self.hidden(X))
        X = self.dropout(X)
        X = F.softmax(self.output(X), dim=-1)
        return X

In [222]:
class Multiclass(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(4, 8)
        self.act = nn.ReLU()
        self.output = nn.Linear(8, 3)
        
    def forward(self, x):
        x = self.act(self.hidden(x))
        x = self.output(x)
        return x

52811

In [223]:
class neuNet(nn.Module):
    def __init__(self):
        super(neuNet,self).__init__()
        self.fc1 = nn.Linear(50, 32)
        self.fc2 = nn.Linear(32,32)
        self.fc3 = nn.Linear(32,3)
        self.droput = nn.Dropout(0.2)

    def forward(self,x):

        x = x.view(-1,50)
        x = F.relu(self.fc1(x))
        x = self.droput(x)
        x = F.relu(self.fc2(x))
        x = self.droput(x)
        x = self.fc3(x)

        return x

In [224]:
model = neuNet().to(device)

In [225]:
model

neuNet(
  (fc1): Linear(in_features=50, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=3, bias=True)
  (droput): Dropout(p=0.2, inplace=False)
)

In [226]:
learning_rate = 0.01
MLP_criterion = nn.CrossEntropyLoss()
Model_optimizer = torch.optim.SGD(model.parameters(),lr =learning_rate)

In [227]:
def accuracy(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    # print(y_pred_tag)
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    # print(correct_results_sum)
    acc = correct_results_sum / y_test.shape[0]
    acc = torch.round(acc * 100)
    return acc

In [228]:
epochs = 50
print_epoch = 5
for epoch in range(epochs):
    iteration_loss = 0.
    iteration_accuracy = 0.
    
    model.train()
    for i, data in enumerate(train_loader):
        X, y = data
        X, y = X.to(device), y.to(device)
        y_pred = model(X.float())
        loss = MLP_criterion(y_pred, y.reshape(-1, 1).float())
        
        iteration_loss += loss
        iteration_accuracy += accuracy(y_pred, y)
        Model_optimizer.zero_grad()
        loss.backward()
        Model_optimizer.step()
    
    if epoch % print_epoch == 0:
        print(f'Train: epoch: {epoch} - loss: {(iteration_loss/(i+1)):.5f}; acc: {(iteration_accuracy/(i+1)):.3f}')
    
    iteration_loss = 0.
    iteration_accuracy = 0.
    model.eval()
    for i, data in enumerate(test_loader):
        X, y = data
        X, y = X.to(device), y.to(device)
        y_pred = model(X.float())
        loss = MLP_criterion(y_pred, y.reshape(-1, 1).float())
        
        iteration_loss += loss
        iteration_accuracy += accuracy(y_pred, y)
        Model_optimizer.zero_grad()
        loss.backward()
        Model_optimizer.step()
    
    if epoch % print_epoch == 0:
        print(f'Test: epoch: {epoch} - loss: {(iteration_loss/(i+1)):.5f}; acc: {(iteration_accuracy/(i+1)):.3f}')
    



RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [236]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.a = df.iloc[:, 0].values
        self.b = df.iloc[:, 1].values
        self.c = df.iloc[:, 2].values
        self.y = df.iloc[:, 3].values
        self.y = list(map(self.string_to_vector, self.y))
        self.length = len(df)

    def string_to_vector(self, value):
        data = {"acute triangle": 0, "right triangle": 1, "obtuse triangle": 2}
        return data.get(value, None)

    def __getitem__(self, index):
        x = torch.FloatTensor(sorted([self.a[index], self.b[index], self.c[index]]))
        y = torch.LongTensor(self.y)[index]
        return x, y

    def __len__(self):
        return self.length


class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()

        self.layer = nn.Sequential(
            nn.Linear(3, 3)
        )

    def forward(self, x):
        x = self.layer(x)
        return x


train_dataset = CustomDataset("../Data/dataset.csv")
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(1000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 100 == 0:
        print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")


with torch.no_grad():
    model.eval()
    classes = {0: "acute triangle", 1: "right triangle", 2: "obtuse triangle"}
    inputs = torch.FloatTensor([
        [9.02, 9.77, 9.96], # 0 | acute triangle
        [8.01, 8.08, 8.32], # 0 | acute triangle
        [3.55, 5.15, 6.26], # 1 | right triangle
        [3.32, 3.93, 5.14], # 1 | right triangle
        [4.39, 5.56, 9.99], # 2 | obtuse triangle
        [3.01, 3.08, 9.98], # 2 | obtuse triangle
        [5.21, 5.38, 5.39], # 0 | acute triangle
        [3.85, 6.23, 7.32], # 1 | right triangle
        [4.16, 4.98, 8.54], # 2 | obtuse triangle
    ]).to(device)
    outputs = model(inputs)
    
    print('---------')
    print(outputs)
    print(torch.round(F.softmax(outputs, dim=1), decimals=2))
    print(outputs.argmax(1))
    print(list(map(classes.get, outputs.argmax(1).tolist())))

Epoch : 1000, Cost : 0.205
Epoch : 2000, Cost : 0.131
Epoch : 3000, Cost : 0.096


KeyboardInterrupt: 

In [1]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.a = df.iloc[:, 0].values
        self.b = df.iloc[:, 1].values
        self.c = df.iloc[:, 2].values
        self.y = df.iloc[:, 3].values
        self.y = list(map(self.string_to_vector, self.y))
        self.length = len(df)

    def string_to_vector(self, value):
        data = {"acute triangle": 0, "right triangle": 1, "obtuse triangle": 2}
        return data.get(value, None)

    def __getitem__(self, index):
        x = torch.FloatTensor(sorted([self.a[index], self.b[index], self.c[index]]))
        y = torch.LongTensor(self.y)[index]
        return x, y

    def __len__(self):
        return self.length

/home/jhmoon/venvFL/env/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataset = CustomDataset("../Data/dataset.csv")

In [4]:
df = pd.read_csv('../Data/dataset.csv')

In [6]:
df.iloc[:, 0].values

array([3.16, 9.46, 7.23, ..., 9.24, 7.86, 6.11])